# Thông tin nhóm 14

1612406 - Đặng Phương Nam

1612423 - Lê Minh nghĩa

# 1. Câu hỏi

Cho các thông tin về căn nhà:

- Giá cho thuê trước khi bán.
- Địa chỉ.
- Loại nhà.
- Số phòng ngủ.
- Số nhà vệ sinh.
- Diện tích phần nhà.
- Diện tích toàn bộ lô đất.
- Năm xây dựng.
- Tiền thuế.
- Gần mấy trường học.
- Tình hình tội phạm.
- ...

Hỏi giá trị của căn nhà là bao nhiêu tiền?

# 2. Lợi ích khi trả lời được câu hỏi

Nhờ vào thông tin của căn nhà:

- Người bán có thể dự đoán được giá trị căn nhà mà mình muốn bán.
- Người mua có thể ước lượng được căn nhà mình muốn mua có giá cả hợp lý hay không?.
- Dự đoán được giá trị căn nhà của mình.
- ...

# 3. Thu thập dữ liệu

## 3.1. Parse HTML

Dữ liệu thu thập từ trang web https://www.realtytrac.com/. Ta chỉ thu thập dữ liệu "các estate đã được bán tại  California", ta có HTML cần parse: https://m.realtytrac.com/mapsearch/sold/ca/

Thời gian lấy dữ liệu: Ngày 10/12/2019

In [ ]:
# Import các thư viện cần thiết.
import urllib.robotparser
import json
import os.path
import re

from requests_html import HTMLSession

In [ ]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://m.realtytrac.com/robots.txt')
rp.read()

In [ ]:
# Lấy link chứa danh sách các ngôi nhà đã bán theo Quận tại California
check_parse = rp.can_fetch('*', 'https://m.realtytrac.com/mapsearch/sold/ca/')

base_url = "https://m.realtytrac.com"
list_url_county = []

# Kiểm tra việc parse HTML được cho phép hay không?
if check_parse == True:
    session = HTMLSession()
    r = session.get('https://m.realtytrac.com/mapsearch/sold/ca/')

    counties = r.html.find("option")

    for county in counties:
        tail_url = county.attrs['value']
        if (tail_url):
            url = base_url + tail_url
            list_url_county.append(url)


list_url_county

In [ ]:
# Hàm trả về generator, generator này trả về trang tiếp theo sau mỗi lần duyệt.
def urlGenerator(baseUrl, startPage=1):
    i = startPage
    if i <= 1:
        yield baseUrl
        i = 2
    while True:
        yield f"{baseUrl}/p-{i}"
        i += 1


# Hàm parseDetailPage: parse để lấy thông tin chi tiết của từng căn nhà từ detailUrl (của căn nhà đã bán).
# Tham số: session và detailUrl của căn nhà cần parse.
# Trả về: dictionary chứa các thông tin đã parse được.
def parseDetailPage(session: HTMLSession, detailUrl: str):

    # Lấy mã HTML của trang web.
    r = session.get(detailUrl)

    # Dictionary lưu lại kết quả.
    result = {}

    # Lấy address của estate.
    address = {}
    street = r.html.find("[itemprop = 'streetAddress']", first=True)
    if street is not None:
        street = street.text

    locality = r.html.find("[itemprop = 'addressLocality']", first=True)
    if locality is not None:
        locality = locality.text

    region = r.html.find("[itemprop = 'addressRegion']", first=True)
    if region is not None:
        region = region.text

    code = r.html.find("[itemprop = 'postalCode']", first=True)
    if code is not None:
        code = code.text

    address["street"] = street
    address["locality"] = locality
    address["region"] = region
    address["code"] = code

    result["address"] = address

    # Lấy thông tin ngày đã bán căn nhà.
    date_sold = r.html.find(".recently-sold", first=True)
    if date_sold is not None:
        result["date_sold"] = date_sold.text.replace("SOLD ON ", "")
    else:
        result["date_sold"] = None

    # Lấy giá cho thuê trước khi bán.
    mortgage = r.html.find(".property-price-info", first=True)
    if mortgage is not None:
        result["mortgage"] = mortgage.text.replace("Est. Mortgage: ","")
    else:
        result["mortgage"] = None


    # Lấy phân mô tả chi tiết về căn nhà.
    details = []
    detailTexts = r.html.find(".detail-text")
    if detailTexts is not None:
        for detailText in detailTexts:
            details.append(detailText.html)  # sở dĩ lấy mà HTML để dễ parse sau này (khi muốn lấy thông tin từ details)
    result["details"] = details

    # Lấy property info của căn nhà.
    propertyInfo = r.html.find(".property-info", first=True)
    if propertyInfo is not None:
        info = {}
        items = propertyInfo.find(".item")
        if items is not None:
            for item in items:
                name = item.find(".name", first=True).text.lower().replace(" ", "_")
                value = item.find(".value", first=True).text
                info[name] = value

        result["info"] = info
    else:
        result["info"] = None

    # Lấy property taxes của căn nhà.
    property_taxes = r.html.find(".property-taxes")
    if property_taxes is not None:
        for tax in property_taxes:
            key = tax.find(".section-head", first=True).text.lower().replace(" ", "_")
            taxes = {}
            items = tax.find(".item")
            if items is not None:
                for item in items:
                    name = item.find(".name", first=True).text.lower().replace(" ", "_")
                    value = item.find(".value", first=True).text
                    taxes[name] = value

            result[key] = taxes

    # Lấy số lượng trường học gần đó.
    local_school = r.html.find(".property-schools", first=True)
    num_school = 0
    if local_school is not None:
        num_school = len(local_school.find(".schoolInfo"))

    result["school"] = num_school

    # Lấy thông tin tội phạm trong vùng.
    crimes = {}
    local_crime_index = r.html.find(".property-local-crime", first=True)
    if local_crime_index is not None:
        type_crimes = local_crime_index.find(".gradeTitle")
        if type_crimes is not None:
            for type_crime in type_crimes:
                match = re.match(r"(.+?) = (\d+)", type_crime.text)
                if match is not None:
                    key = match.group(1).lower().replace(' ', '_')
                    value = int(match.group(2))
                    crimes[key] = value

    result["local_crime_index"] = crimes

    # Lấy số lượng nhà bị tịch thu gần đó.
    nearby_foreclosures = r.html.find(".property-nearby-fc", first=True)
    num_fc = 0
    if nearby_foreclosures is not None:
        num_fc = len(nearby_foreclosures.find(".nearby-property"))

    result["foreclosures"] = num_fc

    return result

In [ ]:
# Hàm parseSearchPage: parse theo page (mỗi parse chứa danh sách và url của 25 căn nhà đã được bán).
# Tham số: session và url của page hiện tại.
# Trả về: list thông tin chi tiết của tất cả căn nhà tại page hiện tại và url của page kế tiếp.
def parseSearchPage(session: HTMLSession, urlPage: str):

    # Lấy mã HTML.
    r = session.get(urlPage)

    # Lấu detailUrl của tất cả bất động sản.
    detailUrls = [link for link in r.html.absolute_links if 'property' in link]

    results = []  # List chứa kết quả parse được tại page.

    # Tiến hành parse từ detailUrl
    for detailUrl in detailUrls:
        # Kiểm tra việc parse HTML được cho phép hay không?
        if not rp.can_fetch('*', detailUrl):
            print(f"SKIP: {detailUrl}")
            continue

        print(f"Parsing detail url {detailUrl}")
        result = parseDetailPage(session, detailUrl)
        results.append(result)

    nextPageAnchor = r.html.find(".current + .page", first=True)
    return results, nextPageAnchor is not None

In [ ]:
# Hàm getAllCountyProperties: parse tất cả các page của một County để lấy thông tin chi tiết của 
# căn nhà được bán thành công.
# Tham số: session và countryUrl chứa thông tin các căn nhà được bán của một County.
# Trả về: File chứa kết quả đã parse thành công (tên file là tên County được parse).
def getAllCountyProperties(session: HTMLSession, countyUrl: str):
    # Lấy tên County từ urlCounty.
    if countyUrl.endswith("/"):
        countyUrl = countyUrl[:-1]
    fileName = f"data/{os.path.basename(countyUrl)}.json"

    # Kiểm tra sự tồn tại file.
    if os.path.exists(fileName):
        print(f"FILE EXISTS: {fileName}")
        return

    # Mở file để ghi dữ liệu (kiểu file là json)
    with open(fileName, "a") as fOut:
        for url in urlGenerator(countyUrl, 1):
            # Kiểm tra việc parse HTML được cho phép hay không?
            if not rp.can_fetch('*', url):
                print(f"SKIP: {url}")
                continue

            # Tiến hành parse theo từng page.
            print(f"Parsing {url}")
            results, nextPage = parseSearchPage(session, url)

            # Ghi kết quả xuống file.
            for result in results:
                json.dump(result, fOut, ensure_ascii=False)
                fOut.write("\n")

            # Kiểm tra có còn page tiếp theo hay không?
            if not nextPage:
                break

            # Mỗi lần parse 1 page thì cho sleep 2s.
            time.sleep(2)

Tiến hành lấy dữ liệu. Lưu ý trong quá trình lấy có thể bị timeout do đường truyền mạng, nếu bị thì chỉ cần làm theo các bước sau:
- Xem đã lấy tới page mấy. vd: "...page-125"
- Đổi lại tên file trong thư mục data là "orange-county-1.json" chẳng hạn (số 1, 2, 3,.. là theo mỗi lần lấy dữ liệu của mình).
- Vào hàm "getAllCountyProperties" đổi lại dòng "for url in urlGenerator(countyUrl, 1)" thành "for url in urlGenerator(countyUrl, 125)", số 125 là tùy mình lấy tới page thứ mấy mà bị timeout.
- Dữ liệu tất nhiên sẽ bi trùng, cứ việc yên tâm vì phần sau sẽ xóa trung.

In [ ]:
url = "https://m.realtytrac.com/mapsearch/sold/ca/orange-county"
session = HTMLSession()
getAllCountyProperties(session, url)

In [ ]:
# Phần gộp từng phần dữ liệu đã lấy dang dở lại thành 1 file.
# Nếu không bị timeout như trường hợp ở trên thì không dùng shell code này.
inputFiles = ["data/orange-county-1.json", "data/orange-county-2.json", 
              "data/orange-county-3.json", "data/orange-county-4.json"]
outputFile = "data/full-orange-county.json"

with open(outputFile, "a+") as fOut:
    for inputFile in inputFiles:
        with open(inputFile, "r") as fIn:
            data = fIn.read()
            fOut.write(data)

In [ ]:
# Phần convert file json sang file csv.

def xstr(s):
    return '' if s is None else str(s)


# Hàm preprocessFile: chuyển file json sang file csv.
# Tham số: tên file json và tên file csv.
# Không trả về.
def preprocessFile(inputFile: str, outputFile: str):

    # Kiểm tra tồn tại file Input.
    if os.path.exists(outputFile):
        print(f"FILE EXISTS: {outputFile}")
        return

    # Kiểm tra tồn tại file Output.
    if not os.path.exists(inputFile):
        print(f"Input file not found: {inputFile}")
        return

    with open(inputFile) as fIn, open(outputFile, "w") as fOut:
        tab = "\t"
        headers = (
            "address_street",
            "address_locality",
            "address_region",
            "address_code",
            "date_sold",
            "mortgage",
            "info_type",
            "info_bedrooms",
            "info_bathrooms",
            "info_size",
            "info_lot_size",
            "info_year_built",
            "info_est_value",
            "info_sold_price",
            "info_property_id",
            "info_county",
            "info_parcel_number",
            "taxes_land",
            "taxes_improvements",
            "taxes_total",
            "taxes_taxes",
            "school",
            "total_crime",
            "violent_crime",
            "property_crime",
            "foreclosures",
        )

        # Ghi các tên cột vào file output, mỗi cột ngăn cách bởi "\t".
        fOut.write(f"{tab.join(headers)}\n")

        # Duyệt từng line trong file Input (json).
        for line in fIn:
            row = json.loads(line)

            # Lấy địa chỉ
            address = row.get("address")
            address_street = address.get("street")
            address_locality = address.get("locality")
            address_region = address.get("region")
            address_code = address.get("code")

            # Lấy ngày bán thành công.
            date_sold = row.get("date_sold")

            # Lấy giá cho thuê mỗi tháng (trước khi được bán).
            mortgage = row.get("mortgage")
            if mortgage is not None:
                mortgage = float(mortgage.replace("$","").replace(",","").replace("/mo","").replace("Est. Refinance: ", ""))

            # Lấy phần info
            info = row.get("info")

            # Type.
            info_type = info.get("type")

            # Bedrooms.
            info_bedrooms = info.get("bedrooms")
            if info_bedrooms == "Contact Agent":
                info_bedrooms = None
            else:
                info_bedrooms = float(info_bedrooms)

            # Bathrooms.
            info_bathrooms = info.get("bathrooms")
            if info_bathrooms == "Contact Agent":
                info_bathrooms = None
            else:
                info_bathrooms = float(info_bathrooms)

            # Size.
            info_size = info.get("size")
            if info_size == "Contact Agent":
                info_size = None
            else:
                info_size = float(info_size.replace(",", "").replace(" sqft", ""))

            # Lot size.
            info_lot_size = info.get("lot_size")
            if info_lot_size == "Contact Agent":
                info_lot_size = None
            else:
                info_lot_size = float(info_lot_size.replace(",", "").replace(" sqft", "").replace(" acres", ""))

            # Year build.
            info_year_built = info.get("year_built")
            if info_year_built  == "Contact Agent":
                info_year_built = None
            else:
                info_year_built = int(info_year_built)

            # Est value.
            info_est_value = info.get("est._value")
            if info_est_value is not None:
                info_est_value = float(info_est_value.replace("$", "").replace(",",""))

            # Sold price.
            info_sold_price = info.get("sold_price")
            if info_sold_price is not None:
                if info_sold_price != "N/A":
                    info_sold_price = float(info_sold_price.replace("$", "").replace(",", ""))

            # ID.
            info_property_id = info.get("property_id")

            # County.
            info_county = info.get("county")

            # Parcel_number.
            info_parcel_number = info.get("parcel_number")

            # Lấy phần Taxes.
            taxes = row.get("property_taxes")
            taxes_land = None
            taxes_improvements = None
            taxes_total = None
            taxes_taxes = None
            if taxes is not None:
                taxes_land = float(taxes.get("land").replace("$", "").replace(",", ""))
                taxes_improvements = float(taxes.get("improvements").replace("$", "").replace(",", ""))
                taxes_total = float(taxes.get("total").replace("$", "").replace(",", ""))
                taxes_taxes = taxes.get("taxes").replace("$", "").replace(",", "")

            # Lấy số lượng scholl gần đó.
            school = int(row.get("school"))

            # Lấy tình hình tội phạm.
            crime = row.get("local_crime_index")
            if crime is not None:
                total_crime = crime.get("total_crime")
                if total_crime is not None:
                    total_crime = int(total_crime)
                violent_crime = crime.get("violent_crime")
                if violent_crime is not None:
                    violent_crime = int(violent_crime)
                property_crime = crime.get("property_crime")
                if property_crime is not None:
                    property_crime = int(property_crime)

            # Lấy số lượng foreclosures gần đó.
            foreclosures = int(row.get("foreclosures"))

            # Row này là tổng hợp các thông tin lấy được ở trên.
            row = (
                xstr(address_street),
                xstr(address_locality),
                xstr(address_region),
                xstr(address_code),
                xstr(date_sold),
                xstr(mortgage),
                xstr(info_type),
                xstr(info_bedrooms),
                xstr(info_bathrooms),
                xstr(info_size),
                xstr(info_lot_size),
                xstr(info_year_built),
                xstr(info_est_value),
                xstr(info_sold_price),
                xstr(info_property_id),
                xstr(info_county),
                xstr(info_parcel_number),
                xstr(taxes_land),
                xstr(taxes_improvements),
                xstr(taxes_total),
                xstr(taxes_taxes),
                xstr(school),
                xstr(total_crime),
                xstr(violent_crime),
                xstr(property_crime),
                xstr(foreclosures),
            )

            # Ghi xuống file output
            fOut.write(f"{tab.join(row)}\n")

&#9889; Sở dĩ nhóm không thêm phần `description` vào dữ liệu là bởi vì khi quan sát phần mã `html` của `description` thì phát hiện ra nó được viết theo một `format` nhất định, các thông tin số liệu của nó được lấy từ các bảng số liệu trong cùng một trang web (tức là các phần thông tin nhà, thuế,.. mà mình đã parse ở trên). Nếu số liệu nào có thì sẽ được lấy ra và tạo một câu mô tả theo `format` cho trước, còn số liệu nào không có thì hiển nhiên không xuất hiện câu đó. Như vậy, có thể kết luận là phần `description` lúc này sẽ không giúp ích gì cho chúng ta.

In [ ]:
inputFile = "data/full-orange-county.json"
outputFile = "data/orange-county.csv"
preprocessFile(inputFile, outputFile)

## 3.2. Lựa chọn dữ liệu của năm 2019 và có "correct ouput"

In [1]:
# Import các thư viện cần thiết.
import pandas as pd

### 3.2.1 Xóa dữ liệu trùng

#### Dữ liệu ban đầu

In [2]:
# Đọc dữ liệu lên
file_csv = "data/Orange-County/orange-county.csv"
data_df =  pd.read_csv(file_csv, sep='\t')
data_df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,info_parcel_number,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,93496030,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4
1,1013 N Mountain View Pl,Fullerton,CA,92831,12/01/1993,3289.0,Single Family Residence,4.0,2.0,2037.0,...,2954406,208547.0,130222.0,338769.0,3944 (1.16 %),3,65.0,49.0,100.0,4
2,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,78623129,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4
3,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,15512543,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4
4,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,93785045,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4
5,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,29247108,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4
6,365 W Skyline Dr,La Habra Heights,CA,90631,05/25/1965,182.0,Single Family Residence,2.0,2.0,1714.0,...,8240004014,56212.0,50609.0,106821.0,2082 (1.94 %),3,35.0,17.0,79.0,4
7,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,50314226,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4
8,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,52719141,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4
9,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,26251319,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4


In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21268 entries, 0 to 21267
Data columns (total 26 columns):
address_street        21204 non-null object
address_locality      21266 non-null object
address_region        21268 non-null object
address_code          21268 non-null int64
date_sold             21268 non-null object
mortgage              21134 non-null float64
info_type             21268 non-null object
info_bedrooms         19148 non-null float64
info_bathrooms        19159 non-null float64
info_size             20282 non-null float64
info_lot_size         14829 non-null float64
info_year_built       19737 non-null float64
info_est_value        1297 non-null float64
info_sold_price       19837 non-null float64
info_property_id      21268 non-null int64
info_county           21268 non-null object
info_parcel_number    21268 non-null int64
taxes_land            20981 non-null float64
taxes_improvements    20981 non-null float64
taxes_total           20981 non-null float64
taxe

Dữ liệu ban đầu có 21268 dòng.

#### Dữ liệu sau khi xóa trùng

In [4]:
# Xóa các phần dữ liệu bị trùng
unique_data_df = data_df.drop_duplicates()

In [5]:
# Xem phần dữ liệu khi đã xóa trùng
unique_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18893 entries, 0 to 21267
Data columns (total 26 columns):
address_street        18841 non-null object
address_locality      18891 non-null object
address_region        18893 non-null object
address_code          18893 non-null int64
date_sold             18893 non-null object
mortgage              18772 non-null float64
info_type             18893 non-null object
info_bedrooms         16966 non-null float64
info_bathrooms        16976 non-null float64
info_size             17993 non-null float64
info_lot_size         13186 non-null float64
info_year_built       17505 non-null float64
info_est_value        1139 non-null float64
info_sold_price       17633 non-null float64
info_property_id      18893 non-null int64
info_county           18893 non-null object
info_parcel_number    18893 non-null int64
taxes_land            18625 non-null float64
taxes_improvements    18625 non-null float64
taxes_total           18625 non-null float64
taxe

Dữ liệu sau khi xóa trùng còn lại 18893 dòng.

In [6]:
# Ghi dữ liệu xuống file csv
unique_data_df.to_csv("data/Orange-County/delete-duplicate-orange.csv", sep='\t', index=False)

### 3.2.2. Lấy dữ liệu của năm 2019

In [7]:
# Đọc dữ liệu lên
df = pd.read_csv("data/Orange-County/delete-duplicate-orange.csv", sep='\t')
df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,info_parcel_number,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,93496030,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4
1,1013 N Mountain View Pl,Fullerton,CA,92831,12/01/1993,3289.0,Single Family Residence,4.0,2.0,2037.0,...,2954406,208547.0,130222.0,338769.0,3944 (1.16 %),3,65.0,49.0,100.0,4
2,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,78623129,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4
3,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,15512543,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4
4,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,93785045,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4
5,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,29247108,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4
6,365 W Skyline Dr,La Habra Heights,CA,90631,05/25/1965,182.0,Single Family Residence,2.0,2.0,1714.0,...,8240004014,56212.0,50609.0,106821.0,2082 (1.94 %),3,35.0,17.0,79.0,4
7,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,50314226,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4
8,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,52719141,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4
9,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,26251319,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18893 entries, 0 to 18892
Data columns (total 26 columns):
address_street        18841 non-null object
address_locality      18891 non-null object
address_region        18893 non-null object
address_code          18893 non-null int64
date_sold             18893 non-null object
mortgage              18772 non-null float64
info_type             18893 non-null object
info_bedrooms         16966 non-null float64
info_bathrooms        16976 non-null float64
info_size             17993 non-null float64
info_lot_size         13186 non-null float64
info_year_built       17505 non-null float64
info_est_value        1139 non-null float64
info_sold_price       17633 non-null float64
info_property_id      18893 non-null int64
info_county           18893 non-null object
info_parcel_number    18893 non-null int64
taxes_land            18625 non-null float64
taxes_improvements    18625 non-null float64
taxes_total           18625 non-null float64
taxe

In [9]:
# Thêm cột năm bán vào dữ liệu
df = df.assign(year_sold=df.date_sold.str[-4:])

In [10]:
# Xem dữ liệu này thu thập của các năm nào
df.year_sold.unique()

array(['2019', '1993', '1965', '1975', '1995', '1979', '1988', '2001',
       '1991', 'SOLD', '2017', '1973', '2005', '1981', '1969', '1989',
       '1976', '1977', '2002', '1984', '1987', '1994', '2003', '1968',
       '2015', '2012', '1971', '2006', '1998', '1967', '2011', '1986',
       '1992', '2018', '1970', '1999', '2000', '1972', '1985', '1974',
       '1978', '1980', '2013', '1996', '2008', '1983', '2007', '2014',
       '2016', '1982', '1997', '1990', '2004', '2010', '1964', '1966',
       '2009', '1963'], dtype=object)

In [11]:
# Chỉ lấy phần dữ liệu của năm 2019
df = df[df.year_sold == "2019"]
df.year_sold = df.year_sold.astype(int)

In [12]:
# Ghi dữ liệu xuống file csv
df.to_csv("data/Orange-County/delete-duplicate-orange-2019.csv", sep='\t', index=False)

### 3.2.3. Lấy dữ liệu của năm 2019 có "correct output"

In [13]:
import pandas as pd

In [14]:
# Đọc dữ liệu
data_df = pd.read_csv("data/Orange-County/delete-duplicate-orange-2019.csv", sep='\t')
data_df.head(10)

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4,2019
1,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4,2019
2,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4,2019
3,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4,2019
4,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4,2019
5,10281 Overhill Dr,Santa Ana,CA,92705,11/22/2019,7268.0,Single Family Residence,5.0,3.0,2814.0,...,525268.0,197448.0,722716.0,8123 (1.12 %),3,128.0,128.0,128.0,4,2019
6,52 Wild Horse,Irvine,CA,92602,11/22/2019,5669.0,Single Family Residence,4.0,3.0,2393.0,...,360108.0,444529.0,804637.0,12770 (1.58 %),3,30.0,22.0,48.0,4,2019
7,8597 Valley View St,Buena Park,CA,90620,11/22/2019,2839.0,Single Family Residence,3.0,2.0,1250.0,...,437818.0,62005.0,499823.0,5871 (1.17 %),3,71.0,40.0,145.0,4,2019
8,13887 La Jolla Plz,Garden Grove,CA,92844,11/22/2019,2226.0,Single Family Residence,3.0,3.0,1637.0,...,332109.0,115034.0,447143.0,5673 (1.26 %),3,76.0,57.0,122.0,4,2019
9,163 Jaripol Cir,Rancho Mission Viejo,CA,92694,11/22/2019,1881.0,Condominium,1.0,2.0,921.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0,2019


In [15]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16011 entries, 0 to 16010
Data columns (total 27 columns):
address_street        15963 non-null object
address_locality      16009 non-null object
address_region        16011 non-null object
address_code          16011 non-null int64
date_sold             16011 non-null object
mortgage              15976 non-null float64
info_type             16011 non-null object
info_bedrooms         14434 non-null float64
info_bathrooms        14444 non-null float64
info_size             15204 non-null float64
info_lot_size         10814 non-null float64
info_year_built       14734 non-null float64
info_est_value        221 non-null float64
info_sold_price       15755 non-null float64
info_property_id      16011 non-null int64
info_county           16011 non-null object
info_parcel_number    16011 non-null int64
taxes_land            15753 non-null float64
taxes_improvements    15753 non-null float64
taxes_total           15753 non-null float64
taxes

Dữ liệu có 16011 căn nhà được bán trong 2019

Ta chỉ lấy dữ liệu có `info_sold_price` khác `null`. Như đã nói ở phần cuối của `1.1. Parse HTML` thì không thể tìm kiếm các giá trị chính xác từ phần `decription` để có thể lắp đầy giá trị `null`, mà nếu có tìm cũng chỉ toàn ra giá trị trung bình của các căn nhà được bán lân cận chứ không phải giá trị chính xác của nó.

In [16]:
df = data_df[pd.notna(data_df["info_sold_price"])]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15755 entries, 0 to 16010
Data columns (total 27 columns):
address_street        15709 non-null object
address_locality      15753 non-null object
address_region        15755 non-null object
address_code          15755 non-null int64
date_sold             15755 non-null object
mortgage              15755 non-null float64
info_type             15755 non-null object
info_bedrooms         14248 non-null float64
info_bathrooms        14258 non-null float64
info_size             14975 non-null float64
info_lot_size         10639 non-null float64
info_year_built       14505 non-null float64
info_est_value        0 non-null float64
info_sold_price       15755 non-null float64
info_property_id      15755 non-null int64
info_county           15755 non-null object
info_parcel_number    15755 non-null int64
taxes_land            15502 non-null float64
taxes_improvements    15502 non-null float64
taxes_total           15502 non-null float64
taxes_t

Dữ liệu còn lại là 15755.

&#9889; Sau khi xóa các dòng có sold_price là `null`. Xem lại cột `info_est_value` thấy toàn `null`, cột này ý nghĩa là giá trị định giá của căn nhà trên các bản rao bán (chưa chốt giá). Nên xóa đi cột này là hoàn toàn hợp lý.

In [18]:
df = df.drop(columns=["info_est_value"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15755 entries, 0 to 16010
Data columns (total 26 columns):
address_street        15709 non-null object
address_locality      15753 non-null object
address_region        15755 non-null object
address_code          15755 non-null int64
date_sold             15755 non-null object
mortgage              15755 non-null float64
info_type             15755 non-null object
info_bedrooms         14248 non-null float64
info_bathrooms        14258 non-null float64
info_size             14975 non-null float64
info_lot_size         10639 non-null float64
info_year_built       14505 non-null float64
info_sold_price       15755 non-null float64
info_property_id      15755 non-null int64
info_county           15755 non-null object
info_parcel_number    15755 non-null int64
taxes_land            15502 non-null float64
taxes_improvements    15502 non-null float64
taxes_total           15502 non-null float64
taxes_taxes           15502 non-null object
scho

In [19]:
# Ghi dữ liệu xuống file csv.
df.to_csv("data/Orange-County/data-2019.csv", sep='\t', index=False)

#### Tách dữ liệu thành 2 tập train và test theo tỉ lệ 80% 20%

In [20]:
# Đọc dữ liệu lên
data_df = pd.read_csv("data/Orange-County/data-2019.csv", sep='\t')
data_df.head()

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
0,4 Silver Crk # 30,Irvine,CA,92603,11/22/2019,3543.0,Condominium,3.0,2.0,1576.0,...,211708.0,157197.0,368905.0,3947 (1.07 %),3,30.0,22.0,48.0,4,2019
1,28459 Alava,Mission Viejo,CA,92692,11/22/2019,2430.0,Single Family Residence,2.0,2.0,1503.0,...,281600.0,148356.0,429956.0,4367 (1.01 %),3,29.0,25.0,39.0,4,2019
2,19441 Hansen Ln,Huntington Beach,CA,92646,11/22/2019,6268.0,Single Family Residence,4.0,3.0,2926.0,...,459321.0,558604.0,1017925.0,11464 (1.12 %),3,57.0,47.0,82.0,4,2019
3,23308 Copante # 103,Mission Viejo,CA,92692,11/22/2019,2158.0,Condominium,2.0,2.0,1308.0,...,326318.0,146682.0,473000.0,4798 (1.01 %),3,29.0,25.0,39.0,4,2019
4,1301 Burwood St,La Habra,CA,90631,11/22/2019,3816.0,Multi-Family Dwellings,NaN,NaN,3756.0,...,233089.0,155289.0,388378.0,5210 (1.34 %),3,55.0,39.0,94.0,4,2019


In [21]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15755 entries, 0 to 15754
Data columns (total 26 columns):
address_street        15709 non-null object
address_locality      15753 non-null object
address_region        15755 non-null object
address_code          15755 non-null int64
date_sold             15755 non-null object
mortgage              15755 non-null float64
info_type             15755 non-null object
info_bedrooms         14248 non-null float64
info_bathrooms        14258 non-null float64
info_size             14975 non-null float64
info_lot_size         10639 non-null float64
info_year_built       14505 non-null float64
info_sold_price       15755 non-null float64
info_property_id      15755 non-null int64
info_county           15755 non-null object
info_parcel_number    15755 non-null int64
taxes_land            15502 non-null float64
taxes_improvements    15502 non-null float64
taxes_total           15502 non-null float64
taxes_taxes           15502 non-null object
scho

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
# Đổi tên
data_df = data_df.rename(columns={"info_sold_price": "sold_price"})

In [24]:
# Tách dữ liệu theo tỉ lệ 80% và 20%
train, test = train_test_split(data_df, test_size=0.2, random_state=0)
train.head().index

Int64Index([10127, 8987, 2064, 13437, 11229], dtype='int64')

&#9889; Nhóm đã cố định `random_state` trong `train_test_split` để đảm bảo kết quả của thầy ra giống với của nhóm. Kết quả của câu lệnh `train.head().index` của nhóm ra 5 giá trị là: [10127, 8987, 2064, 13437, 11229]. 

In [25]:
# Ghi tập train
train.to_csv("data/Orange-County/train-data-2019.csv", sep='\t', index=False)

In [26]:
# Ghi tập test
test.to_csv("data/Orange-County/test-data-2019.csv", sep='\t', index=False)

# 4. Tiền xử lý dữ liệu

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [28]:
data_df = pd.read_csv("data/Orange-County/train-data-2019.csv", sep='\t')
data_df.head()

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
0,15 Via Cima,San Clemente,CA,92672,08/02/2019,9085.0,Single Family Residence,5.0,5.0,3761.0,...,769511.0,755489.0,1525000.0,15444 (1.01 %),3,28.0,18.0,52.0,4,2019
1,28264 Borgona,Mission Viejo,CA,92692,08/16/2019,3270.0,Single Family Residence,2.0,2.0,1503.0,...,388718.0,123031.0,511749.0,5124 (1.00 %),3,29.0,25.0,39.0,4,2019
2,2402 Via Mero,San Clemente,CA,92673,10/31/2019,5496.0,Single Family Residence,4.0,3.0,3339.0,...,496354.0,375945.0,872299.0,8772 (1.00 %),3,28.0,18.0,52.0,4,2019
3,23008 Ditz Ln,Lake Forest,CA,92630,07/01/2019,1819.0,Single Family Residence,2.0,2.0,1180.0,...,208199.0,78817.0,287016.0,2954 (1.02 %),3,29.0,23.0,42.0,4,2019
4,1711 Avenida San Lorenzo,Fullerton,CA,92833,07/24/2019,3943.0,Single Family Residence,3.0,2.0,2151.0,...,621981.0,163064.0,785045.0,8849 (1.12 %),3,65.0,49.0,100.0,4,2019


In [29]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12604 entries, 0 to 12603
Data columns (total 26 columns):
address_street        12567 non-null object
address_locality      12602 non-null object
address_region        12604 non-null object
address_code          12604 non-null int64
date_sold             12604 non-null object
mortgage              12604 non-null float64
info_type             12604 non-null object
info_bedrooms         11360 non-null float64
info_bathrooms        11370 non-null float64
info_size             11948 non-null float64
info_lot_size         8526 non-null float64
info_year_built       11577 non-null float64
sold_price            12604 non-null float64
info_property_id      12604 non-null int64
info_county           12604 non-null object
info_parcel_number    12604 non-null int64
taxes_land            12393 non-null float64
taxes_improvements    12393 non-null float64
taxes_total           12393 non-null float64
taxes_taxes           12393 non-null object
schoo

In [34]:
data_df.describe(include="all")

,address_street,address_locality,address_region,address_code,date_sold,mortgage,info_type,info_bedrooms,info_bathrooms,info_size,...,taxes_land,taxes_improvements,taxes_total,taxes_taxes,school,total_crime,violent_crime,property_crime,foreclosures,year_sold
count,12567,12602,12604,12604.000000,12604,1.260400e+04,12604,11360.000000,11370.000000,11948.000000,...,1.239300e+04,1.239300e+04,1.239300e+04,12393,12604.000000,12207.000000,12207.000000,12207.000000,12604.000000,12604.0
unique,12511,52,2,NaN,160,NaN,13,NaN,NaN,NaN,...,NaN,NaN,NaN,12008,NaN,NaN,NaN,NaN,NaN,NaN
top,1600 S Disneyland Dr,Irvine,CA,NaN,06/28/2019,NaN,Single Family Residence,NaN,NaN,NaN,...,NaN,NaN,NaN,10555 (1.12 %),NaN,NaN,NaN,NaN,NaN,NaN
freq,14,1121,12602,NaN,232,NaN,8225,NaN,NaN,NaN,...,NaN,NaN,NaN,25,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,92555.053713,NaN,9.559392e+03,NaN,3.126849,2.594767,2450.925008,...,5.201897e+05,2.788710e+05,7.990816e+05,NaN,2.877182,54.783976,42.832227,82.494716,3.846715,2019.0
std,NaN,NaN,NaN,560.473700,NaN,8.094353e+04,NaN,1.004007,0.983666,8125.105571,...,1.557344e+06,1.563364e+06,2.848625e+06,NaN,0.594473,29.881531,29.054059,38.045982,0.767292,0.0
min,NaN,NaN,NaN,90620.000000,NaN,2.000000e+00,NaN,1.000000,1.000000,301.000000,...,4.410000e+02,0.000000e+00,0.000000e+00,NaN,0.000000,13.000000,1.000000,23.000000,0.000000,2019.0
25%,NaN,NaN,NaN,92630.000000,NaN,2.553000e+03,NaN,2.000000,2.000000,1265.000000,...,1.513040e+05,7.765000e+04,2.777500e+05,NaN,3.000000,29.000000,22.000000,48.000000,4.000000,2019.0
50%,NaN,NaN,NaN,92677.000000,NaN,3.339000e+03,NaN,3.000000,3.000000,1668.000000,...,3.143840e+05,1.302360e+05,4.753180e+05,NaN,3.000000,55.000000,39.000000,82.000000,4.000000,2019.0
75%,NaN,NaN,NaN,92804.000000,NaN,4.588000e+03,NaN,4.000000,3.000000,2369.000000,...,5.188770e+05,2.335940e+05,7.344000e+05,NaN,3.000000,76.000000,57.000000,122.000000,4.000000,2019.0


## 4.1. Tách tập dữ liệu thành 2 phần train (70%) và validation (30%)

In [30]:
# Tách X và y
y_sr = data_df["info_size"] # sr là viết tắt của series
X_df = data_df.drop("info_size", axis=1)

In [33]:
# Tách tập train và tập validation theo tỉ lệ 70%:30%
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)
train_X_df.head().index

Int64Index([8816, 5891, 4561, 384, 7106], dtype='int64')

&#9889; Nhóm đã cố định `random_state` trong `train_test_split` để đảm bảo kết quả của thầy ra giống với của nhóm. Kết quả của câu lệnh `train_X_df.head().index` của nhóm ra 5 giá trị là: [8816, 5891, 4561, 384, 7106]. 

In [35]:
train_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8822 entries, 8816 to 2732
Data columns (total 25 columns):
address_street        8797 non-null object
address_locality      8821 non-null object
address_region        8822 non-null object
address_code          8822 non-null int64
date_sold             8822 non-null object
mortgage              8822 non-null float64
info_type             8822 non-null object
info_bedrooms         7953 non-null float64
info_bathrooms        7958 non-null float64
info_lot_size         5942 non-null float64
info_year_built       8132 non-null float64
sold_price            8822 non-null float64
info_property_id      8822 non-null int64
info_county           8822 non-null object
info_parcel_number    8822 non-null int64
taxes_land            8680 non-null float64
taxes_improvements    8680 non-null float64
taxes_total           8680 non-null float64
taxes_taxes           8680 non-null object
school                8822 non-null int64
total_crime           856

In [36]:
val_X_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3782 entries, 6041 to 1833
Data columns (total 25 columns):
address_street        3770 non-null object
address_locality      3781 non-null object
address_region        3782 non-null object
address_code          3782 non-null int64
date_sold             3782 non-null object
mortgage              3782 non-null float64
info_type             3782 non-null object
info_bedrooms         3407 non-null float64
info_bathrooms        3412 non-null float64
info_lot_size         2584 non-null float64
info_year_built       3445 non-null float64
sold_price            3782 non-null float64
info_property_id      3782 non-null int64
info_county           3782 non-null object
info_parcel_number    3782 non-null int64
taxes_land            3713 non-null float64
taxes_improvements    3713 non-null float64
taxes_total           3713 non-null float64
taxes_taxes           3713 non-null object
school                3782 non-null int64
total_crime           364

## 4.2. Quan sát và lựa chọn dữ liệu bằng kiến thức cá nhân trên tập train

### 4.2.1. Quan sát phần street

1. Xem thử cột `address_street` có bao nhiêu giá trị thiếu.

In [37]:
len(train_X_df[pd.isna(train_X_df.address_street)])

25

Như vậy có `25` giá trị thiếu.

2. Xem thử có bao nhiêu giá trị khác biệt trong cột `address_street`

In [39]:
len(train_X_df.address_street.unique())

8775

Do dữ liệu ta có `8822` dòng, mà có `25` dòng bị `null`, vì mỗi căn nhà chắc chắn phải có địa chỉ khác nhau nên giá trị này đúng phải là `8822 - 25 + 1 = 8798`. Nhưng tại sau kết quả chạy câu lệnh `len(train_X_df.address_street.unique())` lại ra `8775`?

Ta thử thống kê xem: tại cùng một địa chỉ có bao nhiêu căn bị được bán thành công

In [40]:
address_street_count = train_X_df.address_street.value_counts()
address_street_count

1600 S Disneyland Dr          9
200 W Midway Dr               4
14051 Newport Ave             3
27 Arriate St                 3
8177 Garfield Ave             2
152 Alienta Ln                2
135 Alienta Ln                2
9300 Toledo Way               2
367 Hamilton St               2
148 Alienta Ln                2
20510 Regal Oaks Dr           2
2054 Orange Ave               2
10 Catania                    1
215 S Dove St                 1
6020 E La Paz Way             1
13442 Jessica Dr              1
243 W Canada                  1
25692 Spinnaker Dr            1
2526 Encina Way               1
3520 E Balmoral Dr # 261      1
1533 W Harle Pl               1
7 Roble # 36                  1
6466 Celeste Cir              1
3109 S Griset Pl              1
24386 Hilton Way              1
3133 Bermuda Dr               1
1809 Skyline Dr               1
203 Parkwood                  1
1216 W Sharon Rd              1
22620 Hidden Hills Rd         1
                             ..
25151 Ve

Như quan sát thì có `12 address_treet` có số lần xuất hiện trên 1, trong đó đáng kinh ngạc ở địa chỉ `1600 S Disneyland Dr` lại có số lần xuất hiện lên đến `9` lần. Giờ ta tiên hành lấy các `address_treet` xuất hiện trên 1 lần trong dữ liệu của chúng ta

In [41]:
duplicate_addresss_street = address_street_count[address_street_count > 1].index
duplicate_addresss_street

Index(['1600 S Disneyland Dr', '200 W Midway Dr', '14051 Newport Ave',
       '27 Arriate St', '8177 Garfield Ave', '152 Alienta Ln',
       '135 Alienta Ln', '9300 Toledo Way', '367 Hamilton St',
       '148 Alienta Ln', '20510 Regal Oaks Dr', '2054 Orange Ave'],
      dtype='object')

In [42]:
len(duplicate_addresss_street)

12

### Xem phàn sold_price

In [ ]:
data_df["info_sold_price"].describe()

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
fig = plt.figure()
res = stats.probplot(data_df["info_sold_price"], plot=plt)
plt.show()

### Xem phần street

In [ ]:
data_df[data_df.address_street.str.find("th St") != -1]

In [ ]:
tmp = data_df.address_street.str.replace(r'[^a-zA-Z ]', '').str.strip()

In [ ]:
tmp

In [ ]:
len(tmp.unique())

In [ ]:
df  = data_df[pd.isna(data_df.address_street)]
len(df)

In [ ]:
value_tmp = tmp.value_counts()

In [ ]:
value_tmp 

In [ ]:
#value_tmp["Silver Crk"]
n = 0
for value in value_tmp:
    if value == 1:
        n += 1
n

### Xem phần locality

In [ ]:
list_locality = data_df.address_locality.unique()
print(list_locality)
print(len(list_locality))

In [ ]:
data_df.address_locality.value_counts()

### Xem phần region

In [ ]:
list_region = data_df.address_region.unique()
print(list_region)
print(len(list_region))

In [ ]:
data_df.address_region.value_counts()

### Xem phần address code

In [ ]:
list_address_code = data_df.address_code.unique()
print(list_address_code)
print(len(list_address_code))

In [ ]:
data_df.address_code.value_counts()

### Xem phần county

In [ ]:
list_county = data_df.info_county.unique()
print(list_county)
print(len(list_county))

In [ ]:
data_df.info_county.value_counts()

### Xem phần type của căn nhà

In [ ]:
list_type = data_df.info_type.unique()
print(list_type)
print(len(list_type))

In [ ]:
data_df.info_type.value_counts()

### Xem phần year built 

In [ ]:
list_year_built = data_df.info_year_built.unique()
print(list_year_built)
print(len(list_year_built))

### Xử lý cột taxes_taxes

In [ ]:
tmp = data_df.taxes_taxes.str.partition(' ')

In [ ]:
# Xem phần tiền
tmp[0]

In [ ]:
# Xem phần phần % thuế
tmp[2].str.replace("(","").str.replace(")","") # ảo 2 chỗ ".str" -> why?

### Xem thử xem các id căn nhà có độc lập không

In [ ]:
len(data_df.info_property_id.unique()) # bằng 16011 thì ngon lành!